In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

np.random.seed(24680)

Write your imports in the cell below.

In [5]:
### BEGIN SOLUTION
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
### END SOLUTION

# Ensemble Models and Support Vector Machines Lab
## Training and comparing different algorithms

Once again, we'll work with the bank dataset. This time, the data preprocessing steps have been done for you.

The goal is to try and improve our predictions (if they can be improved at all) using different types of algorithms.

### 1. Read the data (1 point)
This time you only need to read the data. The indicator variables have been separated out for you.

Read the dataset and save it in the variable `bank_data`. The target column is `y`. Use the variables `bank_attributes` and `bank_labels` to save the attributes (explanatory variables, features, predictors), and labels (`y`).

In [6]:
bank_data = None
bank_attributes = None
bank_labels = None
### BEGIN SOLUTION
bank_data = pd.read_csv("data/bank.csv")
bank_attributes = bank_data.drop("y", axis = 1)
bank_labels = bank_data.y
### END SOLUTION

In [7]:
assert_is_not_none(bank_data)
assert_is_not_none(bank_attributes)
assert_is_not_none(bank_labels)
### BEGIN HIDDEN TESTS
assert_equal(bank_data.shape, (4521, 52))
assert_equal(bank_attributes.shape, (4521, 51))
assert_equal(bank_labels.shape, (4521,))
### END HIDDEN TESTS

### 2. Normalize the data (1 point)
Because both forests and SVMs are sensitive to non-scaled data, we need to normalize our dataset first.

Rescale all columns in `bank_attributes` so they have mean = 0 and variance = 1. You can either look at the `sklearn` docs or do this yourself. When you're ready, overwrite the `bank_attributes` column. Make sure that you don't lose the column names in the process.

In [8]:
### BEGIN SOLUTION
scaler = StandardScaler()
bank_attributes[bank_attributes.columns] = scaler.fit_transform(bank_attributes)
### END SOLUTION

In [9]:
assert_is_not_none(bank_attributes)
### BEGIN HIDDEN TESTS
# Check columns are still present
assert_is_not_none(bank_attributes.columns)
assert_equal(len(bank_attributes.columns), 51)
# Check scaling
for mean in bank_attributes.mean():
    assert_almost_equal(mean, 0, delta = 0.001)
for std in bank_attributes.std():
    assert_almost_equal(std, 1, delta = 0.001)
### END HIDDEN TESTS

### 3. Split the data (1 point)
Use the standard 70% / 30% split. Since this is a classification problem, be sure to stratify the split according to the `bank_labels`.

In [10]:
bank_attributes_train, bank_labels_train = None, None
bank_attributes_test, bank_labels_test = None, None
### BEGIN SOLUTION
bank_attributes_train, bank_attributes_test, bank_labels_train, bank_labels_test = train_test_split(
    bank_attributes, bank_labels, train_size = 0.7, test_size = 0.3, stratify = bank_labels)
### END SOLUTION

In [11]:
assert_is_not_none(bank_attributes_train)
assert_is_not_none(bank_labels_train)

assert_is_not_none(bank_attributes_test)
assert_is_not_none(bank_labels_test)
### BEGIN HIDDEN TESTS
assert_equal(bank_attributes_train.shape, (3164, 51))
assert_equal(bank_attributes_test.shape, (1357, 51))
assert_equal(bank_labels_train.shape, (3164,))
assert_equal(bank_labels_test.shape, (1357,))
### END HIDDEN TESTS

### 4. Prepare the cross-validation folds (1 point)
Use a stratified k-fold cross-validation split, with $k = 5$. Fit it to the train data. Save the trained cross-validator to the variable `k_fold`.

The data should already be shuffled. There's no need to shuffle it again.

In [12]:
k_fold = None
### BEGIN SOLUTION
k_fold = StratifiedKFold(n_splits = 5)
### END SOLUTION

In [13]:
assert_is_not_none(k_fold)
### BEGIN HIDDEN TESTS
assert_is_instance(k_fold, StratifiedKFold)
assert_equal(k_fold.n_splits, 5)
assert_equal(k_fold.shuffle, False)
### END HIDDEN TESTS

### 5. Decision Tree (2 points)
Use cross-validation to train and optimize the hyperparameters for a decision tree classifier.

Use grid search with the following grid:
* `max_depth`: 1, 5, 7, 15, 20
* `min_samples_leaf`: 2, 5, 10, 12
* `max_leaf_nodes`: 5, 10, 20

Use the most appropriate scoring metric (remember that accuracy doesn't work in this case because the data is highly imbalanced; we need something which combines precision and recall). Use the cross-validation splits you just created.

Save the grid results in `grid_search`. Save the best classifier in `tree_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [14]:
grid_search = None
tree_classifier = None

### BEGIN SOLUTION
params = {
    "max_depth": [1, 5, 7, 15, 20], 
    "min_samples_leaf": [2, 5, 10, 12],
    "max_leaf_nodes": [5, 10, 20]
}

tree_classifier = DecisionTreeClassifier()
grid_search = GridSearchCV(tree_classifier, params, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
tree_classifier = grid_search.best_estimator_
### END SOLUTION

print("Decision tree; best score:", grid_search.best_score_)

C:\Users\georgi4c\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\georgi4c\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\georgi4c\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\georgi4c\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\georgi4c\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:

Decision tree; best score: 0.5090009764263382


In [14]:
assert_is_not_none(grid_search)
assert_is_not_none(tree_classifier)
### BEGIN HIDDEN TESTS
assert_equal(sorted(grid_search.param_grid.keys()), ["max_depth", "max_leaf_nodes", "min_samples_leaf"])
assert_equal(grid_search.scoring, "f1")
assert_equal(grid_search.cv, k_fold)
assert_is_instance(tree_classifier, DecisionTreeClassifier)
assert_almost_equal(grid_search.best_score_, 0.5090009764263382, delta = 0.001)
assert_equal(tree_classifier, grid_search.best_estimator_)
assert_equal(tree_classifier.max_depth, 15)
assert_equal(tree_classifier.max_leaf_nodes, 10)
assert_equal(tree_classifier.min_samples_leaf, 2)
### END HIDDEN TESTS

### 5. Random Forest (1 point)
Use cross-validation to train and optimize the hyperparameters for a random forest classifier. Use the same technique as before.

Use the following grid:
* `n_estimators`: 100, 200, 300 
* `max_depth`: 20, 50, 100

Note that this grid is on the small side but this is mainly due to performance reasons. Also note that the training will take some time.

Save the grid results in `grid_search`. Save the best classifier in `forest_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

Due to the relatively slow training, we've chosen low values for the parameters. The performance of the random forest will be worse than the decision tree. This is not necessarily the case in general, it's due to the parameters we've chosen to try.

In [15]:
grid_search = None
forest_classifier = None

### BEGIN SOLUTION
params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [20, 50, 100]
}

forest_classifier = RandomForestClassifier()
grid_search = GridSearchCV(forest_classifier, params, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
forest_classifier = grid_search.best_estimator_
### END SOLUTION

print("Random forest; best score:", grid_search.best_score_)

Random forest; best score: 0.392508930899154


In [16]:
assert_is_not_none(grid_search)
assert_is_not_none(forest_classifier)
### BEGIN HIDDEN TESTS
assert_equal(sorted(grid_search.param_grid.keys()), ["max_depth", "n_estimators"])
assert_equal(grid_search.scoring, "f1")
assert_equal(grid_search.cv, k_fold)
assert_is_instance(forest_classifier, RandomForestClassifier)
assert_almost_equal(grid_search.best_score_, 0.392508930899154, delta = 0.001)
assert_equal(forest_classifier, grid_search.best_estimator_)
assert_equal(forest_classifier.max_depth, 50)
assert_equal(forest_classifier.n_estimators, 200)
### END HIDDEN TESTS

### 6. Linear SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for a linear support vector machine. Use the same technique as before.

Use the following grid:
* `C`: 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20

Note that we're choosing relatively small values for `C`. This is allowed because our data is normalized.

Save the grid results in `grid_search`. Save the best classifier in `linear_svm_classifier`. There are many ways to create a linear SVM classifier. Look up the `sklearn` docs to choose the fastest one (in terms of performance).

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [17]:
grid_search = None
linear_svm_classifier = None

### BEGIN SOLUTION
params = {
    "C": [0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20]
}

linear_svm_classifier = LinearSVC()
grid_search = GridSearchCV(linear_svm_classifier, params, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
linear_svm_classifier = grid_search.best_estimator_
### END SOLUTION

print("Linear SVM; best score:", grid_search.best_score_)

C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iteration

Linear SVM; best score: 0.41865943874919853


C:\Users\Yordan\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [18]:
assert_is_not_none(grid_search)
assert_is_not_none(linear_svm_classifier)
### BEGIN HIDDEN TESTS
assert_equal(sorted(grid_search.param_grid.keys()), ["C"])
assert_equal(grid_search.scoring, "f1")
assert_equal(grid_search.cv, k_fold)
assert_is_instance(linear_svm_classifier, LinearSVC)
assert_almost_equal(grid_search.best_score_, 0.41865943874919853, delta = 0.001)
assert_equal(linear_svm_classifier, grid_search.best_estimator_)
assert_equal(linear_svm_classifier.C, 20)
### END HIDDEN TESTS

### 7. Gaussian SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for an SVM with a Gaussian kernel. Use the same technique as before.

Use the following grid:
* `C`: 10, 15, 20, 50, 200
* `gamma`: 0.001, 0.01, 0.1, 0.2

Note that this time we give larger values of `C` because the governing parameter here is `gamma`.

Save the grid results in `grid_search`. Save the best classifier in `gaussian_svm_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [19]:
grid_search = None
gaussian_svm_classifier = None

### BEGIN SOLUTION
params = {
    "C": [10, 15, 20, 50, 200],
    "gamma": [0.001, 0.01, 0.1, 0.2]
}

gaussian_svm_classifier = SVC()
grid_search = GridSearchCV(gaussian_svm_classifier, params, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
gaussian_svm_classifier = grid_search.best_estimator_
### END SOLUTION

print("Gaussian SVM; best score:", grid_search.best_score_)

Gaussian SVM; best score: 0.4576158438127443


In [20]:
assert_is_not_none(grid_search)
assert_is_not_none(gaussian_svm_classifier)
### BEGIN HIDDEN TESTS
assert_equal(sorted(grid_search.param_grid.keys()), ["C", "gamma"])
assert_equal(grid_search.scoring, "f1")
assert_equal(grid_search.cv, k_fold)
assert_is_instance(gaussian_svm_classifier, SVC)
assert_almost_equal(grid_search.best_score_, 0.4576158438127443, delta = 0.001)
assert_equal(gaussian_svm_classifier, grid_search.best_estimator_)
assert_equal(gaussian_svm_classifier.C, 20)
assert_equal(gaussian_svm_classifier.gamma, 0.01)
### END HIDDEN TESTS

### 9. Compare performance on the testing data (1 point)
Now that you've trained all your models, you've got to select the best one. This should be done on the testing data.

Use the appropriate scoring metric to get the testing scores for all your models. Don't forget to pass the **testing**, not the training data. Save all scores.

Choose the best classifier, based on these scores (the one with the highest test score). Of course, this is not enough. We need to look at ROC curves, track performance through other measures, debug the sources of variance in testing results, try more hyperparameters, etc. However, this is enough for an introductory lab :).

Optionally, you can think of combining them into a boosted model but this is out of the scope of this lab.

In [21]:
tree_classifier_score, forest_classifier_score, linear_svm_classifier_score, gaussian_svm_classifier_score = None, None, None, None
### BEGIN SOLUTION
tree_classifier_score = f1_score(bank_labels_test, tree_classifier.predict(bank_attributes_test))
forest_classifier_score = f1_score(bank_labels_test, forest_classifier.predict(bank_attributes_test))
linear_svm_classifier_score = f1_score(bank_labels_test, linear_svm_classifier.predict(bank_attributes_test))
gaussian_svm_classifier_score = f1_score(bank_labels_test, gaussian_svm_classifier.predict(bank_attributes_test))
### END SOLUTION

print("Testing scores:")
print("Decision tree:", tree_classifier_score)
print("Random forest:", forest_classifier_score)
print("Linear SVM:", linear_svm_classifier_score)
print("Gaussian SVM:", gaussian_svm_classifier_score)

Testing scores:
Decision tree: 0.46212121212121204
Random forest: 0.34862385321100914
Linear SVM: 0.3914590747330961
Gaussian SVM: 0.4386617100371747


In [22]:
best_classifier = "" # Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"
### BEGIN SOLUTION
best_classifier = "tree"
### END SOLUTION

In [23]:
assert_not_equal(best_classifier, "")
### BEGIN HIDDEN TESTS
assert_almost_equal(tree_classifier_score, 0.46212121212121204, delta = 0.001)
assert_almost_equal(forest_classifier_score, 0.34862385321100914, delta = 0.001)
assert_almost_equal(linear_svm_classifier_score, 0.3914590747330961, delta = 0.001)
assert_almost_equal(gaussian_svm_classifier_score, 0.4386617100371747, delta = 0.001)
### END HIDDEN TESTS